<a href="https://colab.research.google.com/github/jdspanda/AirTransportNetwork/blob/main/AirTransportNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# "For every minute spent in organizing, an hour is earned." (B.Franklin)
!rm -r sample_data

rm: cannot remove 'sample_data': No such file or directory


In [36]:
# Get files from GitHub repository & importing necesary methods for GitHub integration
!git clone https://github.com/joaquindariosilva/AirTransportNetwork
!pip install python-dotenv
from dotenv import load_dotenv
import os
from google.colab import files
files.upload()
load_dotenv('git_user.env')
GIT_EMAIL = os.getenv('GIT_EMAIL')
GIT_NAME = os.getenv('GIT_NAME')

fatal: destination path 'AirTransportNetwork' already exists and is not an empty directory.
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/d

KeyboardInterrupt: 

In [ ]:
# Installing required packages for the notebook environment
!pip install pandas networkx matplotlib

# Importing libraries for data manipulation and visualization
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# Load CSV files into DataFrame
routes = pd.read_csv('AirTransportNetwork/data/routes.csv')
nodes = pd.read_csv('AirTransportNetwork/data/airports.csv')
airlines = pd.read_csv('AirTransportNetwork/data/airlines.csv')
airports = pd.read_csv('AirTransportNetwork/data/airports.csv')

In [15]:
routes

,index,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [ ]:
from google.colab import drive
drive.mount('/content/drive')